In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import classification_report, balanced_accuracy_score
from sklearn.model_selection import GridSearchCV
import os

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, cohen_kappa_score
from sklearn.model_selection import GroupKFold

In [2]:
#Cargar el archivo que contiene los datos de los tres tipos de espectros
# (Simbioticas-1, Nebulosas-2, Gigantes Rojas-3)
df = pd.read_csv("../../../new/datasets/balanced_grouped.csv")

In [3]:
df

,0,1,2,3,4,5,6,7,8,9,...,335,336,337,338,339,340,341,342,group,id
0,0.015594,0.010444,0.021063,0.025866,0.023642,0.026157,0.032252,0.027204,0.012955,0.010038,...,0.822413,0.854666,0.898153,0.943331,0.963960,1.000000,0.989876,0.990877,9,1
1,0.331350,0.262977,0.212018,0.300518,0.377132,0.383542,0.231755,0.086373,0.090431,0.221658,...,0.622610,0.634173,0.721204,0.828365,0.825327,0.854831,0.801820,0.771081,6,1
2,0.134930,0.075620,0.062616,0.044667,0.121870,0.126496,0.103467,0.070279,0.098824,0.085445,...,0.934640,0.942650,0.890793,0.946226,0.917741,1.000000,0.951573,0.985645,5,1
3,0.109130,0.129643,0.087304,0.051216,0.052661,0.060745,0.052967,0.048967,0.062728,0.064739,...,0.846347,0.877817,0.918180,0.958194,0.972685,1.000000,0.981704,0.972349,7,1
4,0.224262,0.227079,0.146116,0.080234,0.091311,0.125116,0.114054,0.055488,0.044267,0.103987,...,0.032131,0.029224,0.037412,0.051124,0.052330,0.076105,0.075062,0.095761,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0.522966,0.397591,0.313419,0.284905,0.271822,0.246843,0.243718,0.294817,0.378934,0.408562,...,0.707678,0.711110,0.724560,0.741992,0.744172,0.769042,0.766855,0.788741,3,3
2996,0.201920,0.160191,0.144714,0.130479,0.123315,0.146114,0.197060,0.225130,0.199775,0.155762,...,0.681672,0.674718,0.676407,0.681894,0.675200,0.689825,0.683288,0.699186,3,3
2997,0.000000,0.162607,0.413518,0.572730,0.515844,0.394520,0.409770,0.492245,0.425406,0.261534,...,0.840853,0.764674,0.685685,0.607052,0.532962,0.483535,0.458896,0.475790,5,3
2998,0.257628,0.097062,0.009845,0.092240,0.235695,0.254916,0.118284,0.000000,0.053772,0.193354,...,0.975907,0.949945,0.933002,0.920661,0.897001,0.901540,0.889620,0.910721,9,3


In [4]:
# Separamos las características y la variable objetivo
X = df.iloc[:, :-2]
y = df.iloc[:, -1] - 1
group = df.iloc[:, -2]

In [5]:
gkf = GroupKFold(n_splits=10)

# Guardar las métricas para calcular promedios
accuracies = []

# 4. Crear un DataFrame para almacenar las métricas
df_results = pd.DataFrame(columns=['Fold', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Kappa'])

In [6]:
fold_number = 1
for train_index, test_index in gkf.split(X, y, group):
    # print(train_index , len(train_index))
    # print(test_index, len(test_index))
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Creamos el modelo de clasificación de Random Forest
    model = RandomForestClassifier(n_estimators=500, random_state=42)
    # Entrenamos el modelo utilizando el conjunto de entrenamiento
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # predecir las probabilidades para los datos de prueba
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    print(cm)

        # Calcular las métricas
    acc = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    kappa = cohen_kappa_score(y_test, y_pred)

    # Guardar los resultados en el DataFrame
    df_results = df_results.append({
        'Fold': fold_number,
        'Accuracy': acc,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'Kappa': kappa
    }, ignore_index=True)
    fold_number += 1


[[ 86  14   5]
 [  2  94   6]
 [  0   1 100]]


C:\Users\orest\AppData\Local\Temp\ipykernel_12824\1902901598.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append({


[[89  9  2]
 [ 2 84 14]
 [ 4  1 95]]


C:\Users\orest\AppData\Local\Temp\ipykernel_12824\1902901598.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append({


[[98  2  0]
 [ 7 89  4]
 [ 0  9 91]]


...

C:\Users\orest\AppData\Local\Temp\ipykernel_12824\1902901598.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append({


[[ 75  17   7]
 [  0  93   6]
 [  0   0 100]]


C:\Users\orest\AppData\Local\Temp\ipykernel_12824\1902901598.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append({


[[94  5  0]
 [ 2 97  0]
 [ 1  0 98]]


C:\Users\orest\AppData\Local\Temp\ipykernel_12824\1902901598.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append({


In [7]:
df_results

,Fold,Accuracy,Precision,Recall,F1-Score,Kappa
0,1.0,0.909091,0.914182,0.909091,0.908246,0.863749
1,2.0,0.893333,0.895438,0.893333,0.893091,0.840000
2,3.0,0.926667,0.927076,0.926667,0.926477,0.890000
3,4.0,0.923333,0.923895,0.923333,0.922410,0.885000
4,5.0,0.933333,0.936138,0.933333,0.933392,0.900000
5,6.0,0.872910,0.873845,0.872910,0.872848,0.809385
6,7.0,0.916388,0.918529,0.916388,0.914746,0.874606
7,8.0,0.933110,0.936141,0.933110,0.932169,0.899675
8,9.0,0.899329,0.910052,0.899329,0.897137,0.848958
9,10.0,0.973064,0.973351,0.973064,0.973094,0.959596


In [8]:
# Calcular el promedio y la desviación estándar de cada métrica
metrics_mean = df_results[['Accuracy', 'Precision', 'Recall', 'F1-Score', 'Kappa']].mean()
metrics_std = df_results[['Accuracy', 'Precision', 'Recall', 'F1-Score', 'Kappa']].std()


In [26]:
metrics_mean

Accuracy     0.918056
Precision    0.920865
Recall       0.918056
F1-Score     0.917361
Kappa        0.877097
dtype: float64

In [27]:
metrics_std

Accuracy     0.027226
Precision    0.026444
Recall       0.027226
F1-Score     0.027407
Kappa        0.040834
dtype: float64

# Suspected Symbiotic Stars v1

In [12]:
from sklearn.metrics import confusion_matrix

df_sus_sy = pd.read_csv("../../../new/candidate_symbiotic_stars_v1/normalized/Suspected_SY.csv", header=None)

y_pred = rf.predict(df_sus_sy)
cm = confusion_matrix([1 for _ in range(len(df_sus_sy))], y_pred, labels=[1, 2, 3])
print(cm)

[[10  2  3]
 [ 0  0  0]
 [ 0  0  0]]


In [13]:
df_sus_sy_normalized = pd.read_csv("../../../new/candidate_symbiotic_stars_v1/calibrated_data/Suspected_SY.csv")
df_sus_sy_normalized['label'] = y_pred
df5 = df_sus_sy_normalized[['source_id', 'label']]
df5.head(5)

,source_id,label
0,4687286621186701568,1
1,4651824725526390016,1
2,3321366590173335424,3
3,5410876219867043072,1
4,3575939163051304192,1


In [14]:
df6 = pd.read_csv('../../../new/candidate_symbiotic_stars_v1/built_dataset/suspected_SY_dataset.csv')
df6.head(5)

,FIND_NAME,MAIN_ID,OTYPE,SP_TYPE,ID_Gaia,IDS,OTYPES,Gaia DR3
0,RAW 1691,LIN 521,C*,C,Gaia DR2 4687286621186701568,RAW 1691|LIN 521|2MASS J01183570-7242213|OGLE ...,C*|Em*|LP*|LP*|Em*|MIR|NIR|*|C*?|LP?,4687286621186701568
1,[BE74] 583,[BE74] 583,LongPeriodV*,G/Ke:,Gaia DR2 4651824725526390016,2MASS J05265014-7106350|EROS2-star lm058-2n-25...,LP*|Em*|NIR|V*|*,4651824725526390016
2,StHA 55,EM* StHA 55,Mira,NaN,Gaia DR3 3321366590173335424,IRAS 05440+0642|ASAS J054642+0643.7|ASAS J0546...,Mi*|LP*|V*|LP*|SB*|LP*|MIR|V*|Em*|NIR|*|C*?|IR...,3321366590173335424
3,ZZ CMi,V* ZZ CMi,LongPeriodV*,M6I-IIep,Gaia DR3 3155368612444708096,BD+09 1633|AN 306.1934|DO 2156|GCRV 4915|G...,LP*|NIR|V*|*|IR|LP?,3155368612444708096
4,WRAY 16−51,WRAY 16-51,LongPeriodV*,M4,Gaia DR2 5410876219860836224,IRAS 09316-4621|AKARI-IRC-V1 J0933295-463450|D...,LP*|NIR|MIR|Em*|PN|*|IR,5410876219867043072


In [15]:
# Filtro de data frames
df_filtered = df6.merge(df5, left_on=['Gaia DR3'], right_on=['source_id'], how='inner', indicator=True)
df_filtered = df_filtered[df_filtered['_merge'] == 'both']
df_filtered = df_filtered.iloc[:, [0, 1, 2, 7, 9]]
df_filtered.head(5)

,FIND_NAME,MAIN_ID,OTYPE,Gaia DR3,label
0,RAW 1691,LIN 521,C*,4687286621186701568,1
1,[BE74] 583,[BE74] 583,LongPeriodV*,4651824725526390016,1
2,StHA 55,EM* StHA 55,Mira,3321366590173335424,3
3,WRAY 16−51,WRAY 16-51,LongPeriodV*,5410876219867043072,1
4,NSV 05572,V* VX Crv,Mira,3575939163051304192,1


In [16]:
import os

out_name = 'rf_balanced.csv'
out_dir = '../../../new/candidate_symbiotic_stars_v1/output'
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

fullname = os.path.join(out_dir, out_name)
df_filtered.to_csv(fullname, header=True, index=False)

# Suspected Symbiotic Stars v2

In [17]:
from sklearn.metrics import confusion_matrix

df_sus_sy = pd.read_csv("../../../new/candidate_symbiotic_stars_v2/normalized/Suspected_SY.csv", header=None)

y_pred = rf.predict(df_sus_sy)
cm = confusion_matrix([1 for _ in range(len(df_sus_sy))], y_pred, labels=[1, 2, 3])
print(cm)

[[13  2  2]
 [ 0  0  0]
 [ 0  0  0]]


In [18]:
df_sus_sy_normalized = pd.read_csv("../../../new/candidate_symbiotic_stars_v2/calibrated_data/Suspected_SY.csv")
df_sus_sy_normalized['label'] = y_pred
df5 = df_sus_sy_normalized[['source_id', 'label']]
df5.head(5)

,source_id,label
0,6204217186929931520,2
1,4061952680197028224,1
2,670455944074475008,1
3,4068755633500598272,3
4,2060829659152816768,2


In [19]:
df6 = pd.read_csv('../../../new/candidate_symbiotic_stars_v2/built_dataset/suspected_SY_dataset.csv')
df6.head(5)

,FIND_NAME,MAIN_ID,OTYPE,SP_TYPE,ID_Gaia,IDS,OTYPES,Gaia DR3
0,V748 Cen,V* V748 Cen,EclBin,Ae,Gaia DR3 6204217186929931520,CD-32 10517|ALS 18924|CRTS J145936.6-332503|CS...,EB*|Ro*|NIR|V*|Em*|*,6204217186929931520
1,WRAY 16-294,WRAY 16-294,LongPeriodV*,K5,Gaia DR2 4061952680197028224,2MASS J17391381-2538050|DENIS J173913.8-253805...,LP*|PN|NIR|Em*|*|C*?|ISM|LP?,4061952680197028224
2,DASCH J075731.1+201735,ASAS J075731+2017.6,LongPeriodV*,M0III,Gaia DR2 670455944074475008,2MASS J07573112+2017347|ASAS J075731+2017.6|DA...,SB*|LP*|NIR|V*|*|Opt,670455944074475008
3,ASAS J174600-2321.3,ASAS J174600-2321.3,LongPeriodV*_Candidate,F0I,Gaia DR2 4068755633500598272,2MASS J17460018-2321163|ASAS J174600-2321.3|ER...,NIR|V*|*|LP?,4068755633500598272
4,IPHASJ201550.96+373004.2,IRAS 20140+3720,PlanetaryNeb_Candidate,NaN,Gaia DR2 2060829659152816768,2MASS J20155096+3730042|AKARI-IRC-V1 J2015509+...,NIR|*|C*?|IR|LP?|PN?,2060829659152816768


In [20]:
# Filtro de data frames
df_filtered = df6.merge(df5, left_on=['Gaia DR3'], right_on=['source_id'], how='inner', indicator=True)
df_filtered = df_filtered[df_filtered['_merge'] == 'both']
df_filtered = df_filtered.iloc[:, [0, 1, 2, 7, 9]]
df_filtered.head(5)

,FIND_NAME,MAIN_ID,OTYPE,Gaia DR3,label
0,V748 Cen,V* V748 Cen,EclBin,6204217186929931520,2
1,WRAY 16-294,WRAY 16-294,LongPeriodV*,4061952680197028224,1
2,DASCH J075731.1+201735,ASAS J075731+2017.6,LongPeriodV*,670455944074475008,1
3,ASAS J174600-2321.3,ASAS J174600-2321.3,LongPeriodV*_Candidate,4068755633500598272,3
4,IPHASJ201550.96+373004.2,IRAS 20140+3720,PlanetaryNeb_Candidate,2060829659152816768,2


In [21]:
import os

out_name = 'rf_balanced.csv'
out_dir = '../../../new/candidate_symbiotic_stars_v2/output'
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

fullname = os.path.join(out_dir, out_name)
df_filtered.to_csv(fullname, header=True, index=False)

# Other stars

In [14]:
from sklearn.metrics import confusion_matrix

df_sus_sy = pd.read_csv("../../../new/other_stars/normalized/Suspected.csv", header=None)

y_pred = rf.predict(df_sus_sy)
cm = confusion_matrix([1 for _ in range(len(df_sus_sy))], y_pred, labels=[1, 2, 3])
print(cm)

[[1 3 3]
 [0 0 0]
 [0 0 0]]


In [17]:
df_sus_sy_normalized = pd.read_csv("../../../new/other_stars/calibrated_data/Suspected.csv")
df_sus_sy_normalized['label'] = y_pred
df5 = df_sus_sy_normalized[['source_id', 'label']]
df5.head(5)

,source_id,label
0,3321366590173335424,3
1,4557410314849153920,3
2,2022052808961769088,2
3,4052553745525657600,2
4,4050670827750135040,2


In [20]:
df6 = pd.read_csv('../../../new/other_stars/symbad/suspected.csv')
df6.head(5)

,FIND_NAME,MAIN_ID,OTYPE,SP_TYPE,ID_Gaia,IDS,OTYPES,Gaia DR3
0,StHa55,EM* StHA 55,Mira,NaN,Gaia DR3 3321366590173335424,IRAS 05440+0642|ASAS J054642+0643.7|ASAS J0546...,Mi*|LP*|V*|LP*|SB*|LP*|MIR|V*|Em*|NIR|*|C*?|IR...,3321366590173335424
1,GH Gem,V* GH Gem,Symbiotic*,NaN,Gaia DR3 3160625132721733888,SV* SON 3566|AN 241.1943|ATO J106.0532+12.05...,Sy*|V*|NIR|V*|*|Opt,3160625132721733888
2,V503Her,V* V503 Her,Symbiotic*,NaN,Gaia DR3 4557410314849153920,SV* P 4385|AN 170.1936|ASAS J173640+2318.2|A...,LP*|Sy*|V*|Pu*|NIR|V*|*|Opt,4557410314849153920
3,Hen2-442,Hen 2-442,PlanetaryNeb,NaN,Gaia DR2 2022052808961769088,Hen 2-442|AKARI-IRC-V1 J1939433+262933|GSC2 N0...,PN|NIR|MIR|V*|*|IR,2022052808961769088
4,V850 Aql,V* V850 Aql,Symbiotic*,O-rich,Gaia DR2 4263728319553777408,SV* SON 4396|CSV 4646|IRAS 19210+0032|2MASS...,Sy*|Em*|Mas|AB*|LP*|NIR|PN|V*|*|IR|LP?|Mi?,4263728319553777408


In [21]:
# Filtro de data frames
df_filtered = df6.merge(df5, left_on=['Gaia DR3'], right_on=['source_id'], how='inner', indicator=True)
df_filtered = df_filtered[df_filtered['_merge'] == 'both']
df_filtered = df_filtered.iloc[:, [0, 1, 2, 7, 9]]
df_filtered.head(5)

,FIND_NAME,MAIN_ID,OTYPE,Gaia DR3,label
0,StHa55,EM* StHA 55,Mira,3321366590173335424,3
1,V503Her,V* V503 Her,Symbiotic*,4557410314849153920,3
2,Hen2-442,Hen 2-442,PlanetaryNeb,2022052808961769088,2
3,Hen2-379,PN M 1-44,PlanetaryNeb,4052553745525657600,2
4,AS288,PN H 2-43,PlanetaryNeb,4050670827750135040,2


In [22]:
import os

out_name = 'rf_balanced.csv'
out_dir = '../../../new/other_stars/output'
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

fullname = os.path.join(out_dir, out_name)
df_filtered.to_csv(fullname, header=True, index=False)

# Suspected PN

In [34]:
## Suspected PN
eligible_candidates = [
    "IRAS 02379+5724",
    "UCAC2  46104304",
    "IRAS 05495+2620",
    "IRAS 06549-2330",
    "PN G228.1+00.8",
    "PN G239.3-02.7",
    "PN Y-C   40",
    "EGB  2",
    "IRAS 10348-6320",
    "IRAS 11555-6031",
    "IRAS 11415-6540",
    "PN G292.3+00.5",
    "2MASS J05221214-6949580",
    # "[RP2006] J052216-694127",
    # "[RP2006] J052254-694036",
    "[RP2006] J052438-690413",
    "[RP2006]  312"
]

In [35]:
import numpy as np
from sklearn.metrics import confusion_matrix

df_sus_pn = pd.read_csv("../../../new_v3/normalized/NP_Candidate.csv")

# Eliminar la primera columna
df_sus_pn_data = df_sus_pn.drop(df_sus_pn.columns[0], axis=1)

# predecir las probabilidades para los datos de prueba
y_pred = rf.predict(df_sus_pn_data)
cm = confusion_matrix([1 for _ in range(len(df_sus_pn_data))], y_pred, labels=[1,2,3])
print(cm)

[[24 82 40]
 [ 0  0  0]
 [ 0  0  0]]


C:\Users\orest\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:402: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [36]:
df5 = pd.DataFrame(df_sus_pn['source_id'])
df5['label'] = y_pred
df5.head()

,source_id,label
0,457887361083098880,2
1,469648081051557376,2
2,203511119746151040,1
3,3430688759283627776,2
4,2884159294765728768,2


In [37]:
df6 = pd.read_csv('../../../new_v3/symbad/NP_Candidate.csv')
df6

,MAIN_ID,OTYPE,SP_TYPE,ID_Gaia,IDS,OTYPES,RA,DEC,Gaia DR3
0,PN HaWe 3,PlanetaryNeb_Candidate,NaN,Gaia DR2 434853485833190528,PN HaWe 3|2MASS J03163403+4653374|PK 147-09...,NIR|MIR|PN|*|PN?,03 16 34.0393,+46 53 37.297,434853485833190528
1,IRAS 02379+5724,PlanetaryNeb_Candidate,NaN,Gaia DR2 457887361083098880,2MASS J02413591+5737379|IPHAS J024135.93+57373...,NIR|*|IR|PN?,02 41 35.9206,+57 37 38.021,457887361083098880
2,HD 237204,PlanetaryNeb_Candidate,B2III:,Gaia DR2 469648081051557376,HD 237204|AG+56 441|ALS 7847|AP J04002328+56...,Pu*|NIR|MIR|*|IR|PN?|PN?|bC?,04 00 23.2815,+56 54 05.757,469648081051557376
3,UCAC2 46104304,PlanetaryNeb_Candidate,NaN,Gaia DR2 203511119746151040,UCAC2 46104304|IPHAS J044338.64+410941.5|IRAS...,Em*|NIR|*|IR|Opt|PN?,04 43 38.6416,+41 09 41.460,203511119746151040
4,IRAS 05345+2657,PlanetaryNeb_Candidate,NaN,Gaia DR2 3441293067898788992,AKARI-IRC-V1 J0537424+265913|AKARI-FIS-V1 J053...,MIR|*|G|IR|PN?|PN?,05 37 42.3456,+26 59 14.047,3441293067898788992
...,...,...,...,...,...,...,...,...,...
366,IRAS 12481-4903,PlanetaryNeb_Candidate,NaN,Gaia DR2 6079563212927574528,TYC 8253-562-1|AKARI-IRC-V1 J1250577-491951|GA...,MIR|NIR|*|*|G|IR|PN?|PN?,12 50 57.7794,-49 19 52.021,6079563212927574528
367,IRAS 17532-3045,PlanetaryNeb_Candidate,NaN,Gaia DR3 4056100078497762176,UCAC4 297-134949|AKARI-IRC-V1 J1756302-304522|...,MIR|NIR|*|IR|PN?,17 56 30.2700,-30 45 22.394,4056100078497762176
368,IRAS 18321-1401,PlanetaryNeb_Candidate,NaN,Gaia DR2 4104509513893397760,UCAC4 381-119111|AKARI-IRC-V1 J1834570-135848|...,V*|MIR|NIR|*|IR|Opt|PN?|PN?,18 34 56.9988,-13 58 48.905,4104509513893397760
369,IRAS 18340-1302,PlanetaryNeb_Candidate,NaN,Gaia DR3 4104727698224345728,UCAC4 386-108329|AKARI-IRC-V1 J1836531-125922|...,NIR|*|IR|LP?|PN?,18 36 53.0785,-12 59 23.299,4104727698224345728


In [38]:
# Filtro de data frames
df_filtered = df6.merge(df5, left_on=['Gaia DR3'], right_on=['source_id'], how='inner', indicator=True)
df_filtered = df_filtered[df_filtered['_merge'] == 'both']
df_filtered = df_filtered.iloc[:, [0, 1, 2, 6, 7, 8, 10]]
df_filtered

,MAIN_ID,OTYPE,SP_TYPE,RA,DEC,Gaia DR3,label
0,IRAS 02379+5724,PlanetaryNeb_Candidate,NaN,02 41 35.9206,+57 37 38.021,457887361083098880,2
1,HD 237204,PlanetaryNeb_Candidate,B2III:,04 00 23.2815,+56 54 05.757,469648081051557376,2
2,UCAC2 46104304,PlanetaryNeb_Candidate,NaN,04 43 38.6416,+41 09 41.460,203511119746151040,1
3,IRAS 05495+2620,PlanetaryNeb_Candidate,NaN,05 52 42.8154,+26 21 16.018,3430688759283627776,2
4,PN K 2-12,PlanetaryNeb_Candidate,NaN,06 02 06.8725,-37 25 25.207,2884159294765728768,2
...,...,...,...,...,...,...,...
141,NAME PN LDu 26,PlanetaryNeb_Candidate,NaN,20 35 22.7732,+00 12 51.090,4229698606444795392,2
142,WRAY 17-57,PlanetaryNeb_Candidate,NaN,12 53 16.3034,-61 45 26.431,6055339498610714624,3
143,IRAS 18020-3207,PlanetaryNeb_Candidate,NaN,18 05 19.1339,-32 06 59.315,4043047642896333440,2
144,IRAS 12481-4903,PlanetaryNeb_Candidate,NaN,12 50 57.7794,-49 19 52.021,6079563212927574528,2


In [39]:
out_name = 'rf_balanced.csv'
out_dir = '../../../new_v3/output'
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

fullname = os.path.join(out_dir, out_name)
df_filtered.to_csv(fullname, header=True, index=False)

In [40]:
# filter by eligible candidates
eligible_np = df_filtered[df_filtered['MAIN_ID'].isin(eligible_candidates)]
eligible_np = eligible_np.reset_index(drop=True)

out_name = 'rf_balanced_elegible_np_candidates.csv'
out_dir = '../../../new_v3/output'
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

fullname = os.path.join(out_dir, out_name)
eligible_np.to_csv(fullname, header=True, index=False)